# 生产美国标准city county state表

In [10]:
import os
import pandas as pd
from fuzzywuzzy import process

# 文件路径
output_file = 'US_city_county_state_with_standard_county.csv'

# 检查文件是否存在
if os.path.exists(output_file):
    print(f"{output_file} 已经存在，跳过处理。")
else:
    # 读取 CSV 文件
    county_locations = pd.read_csv('county_locations.csv')
    us_city_county_state = pd.read_csv('US_city_county_state.csv')

    # 创建一个新的列用于存储标准的County名称
    us_city_county_state['StandardCountyName'] = us_city_county_state['CountyName']

    # 定义一个函数用于模糊匹配
    def fuzzy_match_county(county_name, possible_counties):
        # 获取匹配结果
        match, score = process.extractOne(county_name, possible_counties)
        # 如果匹配分数大于等于80%，返回匹配的名称，否则返回None
        return match if score >= 80 else None

    # 遍历US_city_county_state中的每一行，进行模糊匹配
    for idx, row in us_city_county_state.iterrows():
        state_name = row['StateName']
        county_name = row['CountyName']

        # 获取 county_locations 中同一州(State)下的County列表
        possible_counties = county_locations[county_locations['State'] == state_name]['County'].tolist()

        # 如果有可能的County进行模糊匹配
        if possible_counties:
            best_match = fuzzy_match_county(county_name, possible_counties)

            # 如果找到最佳匹配，替换CountyName为标准County名称
            if best_match:
                us_city_county_state.at[idx, 'StandardCountyName'] = best_match

    # 输出处理后的结果
    us_city_county_state.to_csv(output_file, index=False)

    print(f"匹配完成，结果已保存到 '{output_file}'")


US_city_county_state_with_standard_county.csv 已经存在，跳过处理。


# 地名匹配

In [49]:
import pandas as pd
import os
from rapidfuzz import process, fuzz  # 从rapidfuzz直接导入fuzz

# 定义相似度匹配函数，优先匹配城市，无法匹配城市时匹配CountyName，返回StandardCountyName
def match_location(des_vals, matched_state, us_counties_df, location_cache):
    state_df = us_counties_df[us_counties_df['StateName'] == matched_state]
    
    for des in des_vals:
        if pd.notna(des):
            # 检查缓存
            if (matched_state, des) in location_cache:
                return location_cache[(matched_state, des)]

            # 优先匹配城市
            match_city = process.extractOne(des, state_df['City'], scorer=fuzz.ratio)
            if match_city is not None:
                city, similarity = match_city[0], match_city[1]
                if similarity >= 80:
                    # 获取对应的StandardCountyName
                    standard_county_name = state_df[state_df['City'] == city]['StandardCountyName'].values[0]
                    location_cache[(matched_state, des)] = standard_county_name  # 缓存结果
                    return standard_county_name

            # 如果无法匹配到城市，匹配CountyName
            match_county = process.extractOne(des, state_df['CountyName'], scorer=fuzz.ratio)
            if match_county is not None:
                county, similarity = match_county[0], match_county[1]
                if similarity >= 80:
                    # 获取对应的StandardCountyName
                    standard_county_name = state_df[state_df['CountyName'] == county]['StandardCountyName'].values[0]
                    location_cache[(matched_state, des)] = standard_county_name  # 缓存结果
                    return standard_county_name
    
            # 如果无法匹配到城市和CountyName，直接匹配StandardCountyName
            match_standard = process.extractOne(des, state_df['StandardCountyName'], scorer=fuzz.ratio)
            if match_standard is not None:
                standard_county_name, similarity = match_standard[0], match_standard[1]
                if similarity >= 80:
                    location_cache[(matched_state, des)] = standard_county_name  # 缓存结果
                    return standard_county_name    

    return None

# 加载标准地名库
us_counties_df = pd.read_csv('US_city_county_state_with_standard_county.csv')

# 文件路径
input_folder = './dataproducts'
output_folder = './dataproducts2'

# 确保输出文件夹存在
os.makedirs(output_folder, exist_ok=True)

# 列出文件夹中所有的xlsx文件并按字母顺序排序
xlsx_files = sorted([file for file in os.listdir(input_folder) if file.endswith('.xlsx')])

# 定义缓存字典
location_cache = {}

# 逐个处理每个xlsx文件
for file in xlsx_files:
    state_name = os.path.splitext(file)[0]  # 从文件名提取州名称
    output_file_path = os.path.join(output_folder, f"{state_name}.xlsx")
    
    # 检查输出文件是否已经存在
    if os.path.exists(output_file_path):
        print(f"{state_name} 已经处理，跳过...")
        continue

    print(f"正在处理州：{state_name}")
    
    file_path = os.path.join(input_folder, file)
    df = pd.read_excel(file_path)
    
    # 获取有效的Des列（动态检查哪些Des列存在）
    des_columns = [col for col in ['Des_3', 'Des_4', 'Des_5', 'Des_6'] if col in df.columns]
    
    # 为每一行数据匹配城市或县，并返回StandardCountyName
    for idx, row in df.iterrows():
        # 从文件名确定州
        matched_state = state_name  # 文件名即州名
        
        # 获取存在的Des列的值
        des_vals = [row[col] for col in des_columns if pd.notna(row[col])]
        
        # 尝试匹配城市或县，返回StandardCountyName
        matched_location = match_location(des_vals, matched_state, us_counties_df, location_cache)
        
        # 只有当Matched_State和Matched_Location同时存在时才写入
        if matched_state and matched_location:
            df.at[idx, 'Matched_State'] = matched_state
            df.at[idx, 'Matched_Location'] = matched_location
        else:
            df.at[idx, 'Matched_State'] = None
            df.at[idx, 'Matched_Location'] = None

    # 保存处理后的结果到输出文件夹，以文件名命名
    df.to_excel(output_file_path, index=False)
    print(f"已处理并保存文件：{output_file_path}")


正在处理州：Alabama
已处理并保存文件：./dataproducts2\Alabama.xlsx
正在处理州：Alaska
已处理并保存文件：./dataproducts2\Alaska.xlsx
正在处理州：Arizona
已处理并保存文件：./dataproducts2\Arizona.xlsx
正在处理州：Arkansas
已处理并保存文件：./dataproducts2\Arkansas.xlsx
正在处理州：California
已处理并保存文件：./dataproducts2\California.xlsx
正在处理州：Colorado
已处理并保存文件：./dataproducts2\Colorado.xlsx
正在处理州：Connecticut
已处理并保存文件：./dataproducts2\Connecticut.xlsx
正在处理州：Delaware
已处理并保存文件：./dataproducts2\Delaware.xlsx
正在处理州：District of Columbia
已处理并保存文件：./dataproducts2\District of Columbia.xlsx
正在处理州：Florida
已处理并保存文件：./dataproducts2\Florida.xlsx
正在处理州：Georgia
已处理并保存文件：./dataproducts2\Georgia.xlsx
正在处理州：Hawaii
已处理并保存文件：./dataproducts2\Hawaii.xlsx
正在处理州：Idaho
已处理并保存文件：./dataproducts2\Idaho.xlsx
正在处理州：Illinois
已处理并保存文件：./dataproducts2\Illinois.xlsx
正在处理州：Indiana
已处理并保存文件：./dataproducts2\Indiana.xlsx
正在处理州：Iowa
已处理并保存文件：./dataproducts2\Iowa.xlsx
正在处理州：Kansas
已处理并保存文件：./dataproducts2\Kansas.xlsx
正在处理州：Kentucky
已处理并保存文件：./dataproducts2\Kentucky.xlsx
正在处理州：Louisiana
已处理并保存文件：./dat